In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from functions import loading, Adams_wedge_thickness, wedgeold_thickness
import os
from scipy import ndimage
import pandas as pd


In [ ]:
save_folder012 = '/Volumes/KasiaDrive/Processed_012'
files012 = os.listdir(save_folder012)
save_folder_art = '/Volumes/KasiaDrive/Processed_Artemis'
filesart = os.listdir(save_folder_art)

In [ ]:
filesart

In [ ]:
chirp_012 = loading(save_folder012, "Scan_220214_('MgO400',)um_1800nm_27gain_10000ms_finerepeat_1750_3950V_1cm.npz")
chirp_Artemis = loading(save_folder_art, "Scan_220131_('MgO200',)um_1800_shortnm_rotation0_iris-26.5_120000MCP.npz")

In [ ]:
df = pd.DataFrame()
with np.load(os.path.join(save_folder012, 'Scan_210218_MgO400um_1800nm_1cm.npz'), allow_pickle=True) as f:
    df['wedge'] = f['wedge']
    df['data'] = f['data']
    df['eV'] = f['eV']
chirp_old  = df

In [ ]:
#Checking for chirp changes
scan = chirp_old
N = len(scan.wedge.unique())
fig, ax = plt.subplots(N//2+1,2, figsize=(6, N), sharex=True)
ax = ax.flatten()
for i in range(N):
    ax[i].imshow(scan[scan['wedge']==scan.wedge.unique()[i]].data.sum(), aspect='auto', vmax=0.01)
    ax[i].set_ylabel('Wedge %.1f' %scan.wedge.unique()[i])
    ax[i].set_yticks([])
    #ax[i].set_xticks([])
plt.tight_layout()
plt.show()

In [ ]:
#Checking for chirp changes
scan = chirp_Artemis
N = len(scan.wedge.unique())
fig, ax = plt.subplots(N//2+1,2, figsize=(6, N), sharex=True)
ax[0][0].set_title('Iris %.f Lens %.f'%(scan.iris.unique()[0],scan.Lens.unique()[0]))
ax = ax.flatten()
for i in range(N):
    ax[i].imshow(scan[scan['wedge']==scan.wedge.unique()[i]].data.sum(), aspect='auto')
    ax[i].set_ylabel('Wedge %.1f' %scan.wedge.unique()[i])
    ax[i].set_yticks([])
    #ax[i].set_xticks([])
plt.tight_layout()
plt.show()

In [ ]:
scan = chirp_012
scan = scan[scan['iris']==scan.iris.unique()[-1]]
N = len(scan.wedge.unique())
fig, ax = plt.subplots(N//2+1,2, figsize=(6, N), sharex=True)
ax[0][0].set_title('Iris %.f Lens %.f'%(scan.iris.unique()[0],scan.Lens.unique()[0]))
ax = ax.flatten()
for i in range(N):
    ax[i].imshow(scan[scan['wedge']==scan.wedge.unique()[i]].data.sum(), vmax=4,aspect='auto')
    ax[i].set_ylabel('Wedge %.1f' %scan.wedge.unique()[i])
    ax[i].set_yticks([])
    #ax[i].set_xticks([])
plt.tight_layout()
plt.show()

In [ ]:
chirp_copy = chirp_012[chirp_012['iris']==chirp_012.iris.unique()[-1]]
max(chirp_copy[chirp_copy['wedge']==chirp_copy.wedge.unique()[11]].data.sum().flatten())


In [ ]:
chirp_copy = chirp_012[chirp_012['iris']==chirp_012.iris.unique()[-1]].copy()
trace_012 = chirp_copy[chirp_copy['wedge']==chirp_copy.wedge.unique()[11]].data.sum()[:,500:3000]
plt.imshow(trace_012, vmax=10)
plt.figure()
plt.hist(trace_012.flatten(), bins=255)
plt.ylim(0.1)
plt.yscale('log')



In [ ]:
gauss_denoised = ndimage.gaussian_filter(trace_012, 6)
plt.imshow(gauss_denoised, vmax=6)
plt.colorbar()
plt.figure()
plt.hist(gauss_denoised.flatten(), bins=255)
plt.ylim(0.1)
plt.yscale('log')

In [ ]:
trace_old = chirp_old[chirp_old['wedge']==chirp_old.wedge.unique()[7]].data.sum()[:,500:3000]
plt.imshow(trace_012, vmax=10)
plt.figure()
plt.hist(trace_012.flatten(), bins=255)
plt.ylim(0.1)
plt.yscale('log')


In [ ]:
wedge012 = chirp_012.wedge.unique()[11]
chirp_copy = chirp_012[chirp_012['iris']==chirp_012.iris.unique()[-1]].copy()
trace_012 = chirp_copy[chirp_copy['wedge']==wedge012].data.sum()[:,500:3000]
trace_012 = ndimage.gaussian_filter(trace_012, 6)
norm_012 = max(trace_012.flatten())
trace_012/norm_012
eV_012 = chirp_copy[chirp_copy['wedge']==wedge012].eV.values[0][500:3000]
wedgeold = chirp_old.wedge.unique()[7]
trace_old = chirp_old[chirp_old['wedge']==wedgeold].data.sum()
trace_old = ndimage.gaussian_filter(trace_old, 6)
trace_old/max(trace_old.flatten())
eV_old = chirp_old[chirp_old['wedge']==wedgeold].eV.values[0]
wedgeart = 2750
trace_art = chirp_Artemis[chirp_Artemis['wedge']==wedgeart].data.sum()
norm_art = max(trace_art.flatten())
trace_art = trace_art/norm_art
eV_art = chirp_Artemis[chirp_Artemis['wedge']==wedgeart].eV.values[0]

In [ ]:
fig, ax = plt.subplots(2, sharex=True)
ax[0].contourf(eV_012, np.arange(trace_012.shape[0])[300:575], trace_012[300:575,:], levels=25, cmap='magma')
ax[0].set_yticks([])
ax[1].contourf(eV_art, np.arange(trace_art.shape[0]), trace_art, levels=50, cmap='magma')
ax[1].set_yticks([])
ax[1].set_xlim(14,22)
har_ord = np.array(range(45))[11:34:2]
wl = 1750
for i in har_ord:
    ax[1].axvline((1240/wl)*i, color='white')
    ax[0].axvline((1240/wl)*i, color='white')
    if (1240/1800)*i >= 14 and (1240/1800)*i <= 22:
        ax[0].text((1240/wl)*i,575, 'H{}'.format(i))

wl = 1800
for i in har_ord:
    ax[1].axvline((1240/wl)*i, color='red')
    ax[0].axvline((1240/wl)*i, color='red')


    

In [ ]:
har_ord

In [ ]:
np.array(range(30))[12:29:2]

In [ ]:
scan = chirp_012
scan = scan[scan['iris']==scan.iris.unique()[-1]]
N = len(scan.wedge.unique())
fig, ax = plt.subplots(1,N-4, sharey=True, figsize=(N,5))
for i in range(N)[4:]:
    eV_012 = scan[scan['wedge']==scan.wedge.unique()[i]].eV.values[0]
    mask_eV = (eV_012>16)*(eV_012<19.5)
    trace_012 = scan[scan['wedge']==scan.wedge.unique()[i]].data.sum()
    trace_012 = ndimage.gaussian_filter(trace_012, 5)
    trace_012/norm_012
    ax[i-4].contourf(np.arange(trace_012.shape[0])[300:575], eV_012[mask_eV], trace_012[300:575,mask_eV].T, cmap='magma')
    ax[i-4].set_xlabel('%.1f' %wedgeold_thickness(scan.wedge.unique()[i]), size=14)
    ax[i-4].set_yticks([])
    ax[i-4].set_xticks([])


ax[0].set_ylim(16,19.5)
ax[0].set_ylabel('Energy [eV]', size=18)
ax[0].set_yticks([16,17,18,19])
ax[0].tick_params(axis='y', labelsize=14)
plt.subplots_adjust(wspace = .001)
plt.show()

In [ ]:
scan = chirp_Artemis
scan = scan[scan['iris']==scan.iris.unique()[-1]]
N = len(scan.wedge.unique())
fig, ax = plt.subplots(1,N, sharey=True, figsize=(N,5))
for i in range(N):
    trace_art = scan[scan['wedge']==scan.wedge.unique()[i]].data.sum()
    trace_art/norm_art
    eV_art = scan[scan['wedge']==scan.wedge.unique()[i]].eV.values[0]
    ax[i].contourf(np.arange(trace_art.shape[0]), eV_art, trace_art.T, cmap='magma', levels=25)
    ax[i].set_xlabel('%.1f' % Adams_wedge_thickness(scan.wedge.unique()[i]), size=14)
    ax[i].set_yticks([])
    ax[i].set_xticks([])


ax[0].set_ylim(14,22)
ax[0].set_ylabel('Energy [eV]', size=18)
ax[0].set_yticks([14,16,18,20,22])
ax[0].tick_params(axis='y', labelsize=14)
plt.subplots_adjust(wspace = .001)
plt.show()

In [ ]:
Adams_wedge_thickness